## Saving model results to a RDBMS

#### Saving the predictions of a model to SQLite

#### Tags:
    Data: labeled data, Kaggle competition
    Technologies: python, pandas, sqlite
    Techniques: Making predictions, storing and retrieving data in a relational database

#### Resources:
[Python - SQLite](https://pythonspot.com/python-database-programming-sqlite-tutorial/)

[SQLite Homepage](https://sqlite.org/about.html)

[Kaggle competition data - Graduate admissions](https://www.kaggle.com/mohansacharya/graduate-admissions)

In [196]:
import pandas as pd
import sqlite3 as lite
import os

import pickle
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


## Overview of the process

Here are the rough steps to create the model and save the model results to  SQL lite DB.

##### A. Building the model and preparing it for later use
    1. Build the model
    2. Store the model to disk

##### B. Preparing the database with *student_score* and *predicted_probability_of_admission* table
    1. create the SQL lite db
    2. create the tables *student_scores* and *predcited_probability_of_admission*

##### C. Using the model on a daily basis to create the predictions on newly added students 
    1. Open a database connection and read the *student_scores* table
    2. Open the saved scaling and modeling objects
    3. Scale the new data and use the model for prediction
    4. Store the results in the *predicted_probability_of_admission*


### Building the model

We start the process by building the model to predict if a student will be accepted at the university given the values of their scores and research done.


In [197]:
'''
With pandas we can import the data from e.g. a .csv file and then store it into a sqlite db table. 
'''

df = pd.read_csv('../../data/Admission_Predict_Ver1.1.csv')
# Data taken from: https://www.kaggle.com/xvivancos/barcelona-data-sets#births.csv

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
Serial No.           500 non-null int64
GRE Score            500 non-null int64
TOEFL Score          500 non-null int64
University Rating    500 non-null int64
SOP                  500 non-null float64
LOR                  500 non-null float64
CGPA                 500 non-null float64
Research             500 non-null int64
Chance of Admit      500 non-null float64
dtypes: float64(4), int64(5)
memory usage: 35.2 KB


In [198]:
df.drop(['Serial No.'],axis=1,inplace=True)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [199]:
# We will take first 450 entries to train the model and the last 50 to use later in model predictions
df_last_50 = df[-50:]
df_last_50.drop(['Chance of Admit '], axis=1, inplace=True)

df = df[:-50]
df

/home/spavko/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
6,321,109,3,3.0,4.0,8.20,1,0.75
7,308,101,2,3.0,4.0,7.90,0,0.68
8,302,102,1,2.0,1.5,8.00,0,0.50
9,323,108,3,3.5,3.0,8.60,0,0.45


In [200]:
X = df.drop(['Chance of Admit '],axis=1)
y = df['Chance of Admit ']

In [201]:
def toFloat64(df):
    for column in df.columns:
        if df[column].dtypes == 'int64':
            df[column] = df[column].astype('float64')
    
    return df   

In [202]:
# The conversion of all the int columns to float would happen anyway because of the scaling process
# that follows
X = toFloat64(X)
            
# we can use sikict-learn libarary to scale the data
scaler = StandardScaler()
scaler.fit(X)

# store the scaler object for later use
dump(scaler, 'scaler.joblib')

# return the scaling results into a new variable
X_scaled = scaler.transform(X)

# fit_transform returns an array, hence i convert it back to df
X_scaled = pd.DataFrame(X_scaled,columns=X.columns)
X_scaled.info()

# import the libraries needed for Linear Regression with sklearn and fit the model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_scaled,y)

# serialize and store the model for later use
path = os.getcwd() + '/serialized_LR.pkl'
with open(path, 'wb') as f:
    pickle.dump(lr, f)
    print("Pickled LR at {}".format(path))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 7 columns):
GRE Score            450 non-null float64
TOEFL Score          450 non-null float64
University Rating    450 non-null float64
SOP                  450 non-null float64
LOR                  450 non-null float64
CGPA                 450 non-null float64
Research             450 non-null float64
dtypes: float64(7)
memory usage: 24.7 KB
Pickled LR at /home/spavko/Desktop/CVs/Interview_Prep/project-portfolio/sasa-pavkovic-portfolio/deploying-model/model-results-to-rdbms/serialized_LR.pkl


In [203]:
lr.score(X_scaled,y)

0.8164809265435556

### Creating the RDBMS environement

We create the later needed tables. 

In [204]:
# Connect to the db
con = lite.connect('uni_admissions.db')
cur = con.cursor()

df_last_50.to_sql('student_scores', con)


/home/spavko/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [205]:
cur.execute('SELECT * from student_scores')

In [206]:
# List of column names
colnames = [description[0] for description in cur.description]
colnames

['index',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research']

In [207]:
student_scores = pd.read_sql_query('select * from student_scores',con)

In [208]:
student_scores

,index,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,450,320,112,4,3.0,4.5,8.86,1
1,451,324,113,4,4.5,4.5,9.25,1
2,452,328,116,4,5.0,3.5,9.60,1
3,453,319,103,3,2.5,4.0,8.76,1
4,454,310,105,2,3.0,3.5,8.01,0
5,455,305,102,2,1.5,2.5,7.64,0
6,456,299,100,2,2.0,2.0,7.88,0
7,457,295,99,1,2.0,1.5,7.57,0
8,458,312,100,1,3.0,3.0,8.53,1
9,459,329,113,4,4.0,3.5,9.36,1


In [209]:
# create the vectors of explanatory variables
s_scores = student_scores.drop(['index'],axis=1)

# load the scaler object
s = load('scaler.joblib')

# scale the new test data and convert to df
s_scores_scaled = s.transform(s_scores)
s_scores_scaled = pd.DataFrame(s_scores_scaled,columns=s_scores.columns)

# load the model
path = os.getcwd() + '/serialized_LR.pkl'
with open(path, 'rb') as f:
    loaded_model = pickle.load(f)
    
# make the predicitons with the model on the scaled data
s_scores_prediction = loaded_model.predict(s_scores_scaled)
s_scores_prediction

/home/spavko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


array([0.81089689, 0.86665306, 0.9032532 , 0.75687495, 0.61692785,
       0.53526503, 0.53717056, 0.47438201, 0.67709675, 0.86843428,
       0.76413442, 0.6038317 , 0.60222971, 0.58755089, 0.46654639,
       0.64356693, 0.74736494, 0.78141024, 0.8224773 , 0.84165807,
       0.84836325, 0.63902319, 0.90883602, 0.63510955, 0.61759487,
       0.55552354, 0.59072497, 0.61348184, 0.73130292, 0.81430765,
       0.78695526, 0.71414444, 0.77411636, 0.59165475, 0.63425514,
       0.6708286 , 0.65248185, 0.83193686, 0.77005475, 0.70104088,
       0.66923212, 0.56229023, 0.59939814, 0.57751426, 0.63422383,
       0.83315032, 0.99097455, 0.94988858, 0.70579776, 0.8236348 ])

In [210]:
# store the results of the model and prepare for creating the table in db
result = pd.DataFrame(list(zip(student_scores['index'].values,s_scores_prediction)),columns=['index','predicted_probability'])
result

,index,predicted_probability
0,450,0.810897
1,451,0.866653
2,452,0.903253
3,453,0.756875
4,454,0.616928
5,455,0.535265
6,456,0.537171
7,457,0.474382
8,458,0.677097
9,459,0.868434


In [211]:
# store the result as a new table in the db
result.to_sql('predicted_probability_of_admission', con)

In [216]:
# Finally, the end user can now combine the 2 tables to retrieve the results of the model predcitions

sql = '''SELECT 
            student_scores."index"
            , predicted_probability_of_admission."predicted_probability"
        FROM student_scores 
        INNER JOIN predicted_probability_of_admission on student_scores."index" = predicted_probability_of_admission."index"
        ;'''

student_predicted_scores = pd.read_sql_query(sql,con)
student_predicted_scores

,index,predicted_probability
0,450,0.810897
1,451,0.866653
2,452,0.903253
3,453,0.756875
4,454,0.616928
5,455,0.535265
6,456,0.537171
7,457,0.474382
8,458,0.677097
9,459,0.868434


In [218]:
# Delete the database so we can repeat the process again next times
try:
    os.remove('uni_admissions.db')
    print('Temporary database deleted!')
except OSError:
    pass


### Final thoughts

The process of updating the predictions can be automatized by a scheduler that would run once per day and insert new entries. Hence the data about predictions would be available at the same time the new data is added to DWH, ensuring seamless integration. 